In [1]:
import pandas as pd
import numpy as np
import re
from typing import Optional

df = pd.read_csv('preprocessedPhase1FoodFacts.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")

Dataset shape: (5651, 47)

Column names:
['url', 'product_name', 'barcode', 'brand', 'quantity', 'serving_size', 'nutriscore_letter', 'nova_group', 'ingredients_text', 'allergens', 'traces', 'energy_kcal_100g', 'fat_100g', 'saturated_fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'main_image_url', 'categories', 'contains_palm_oil', 'vegetarian_status', 'vegan_status', 'nutrient_level_fat', 'nutrient_level_saturated_fat', 'nutrient_level_sugars', 'nutrient_level_salt', 'additives', 'packaging', 'stores', 'countries', 'origins', 'manufacturing_places', 'ecoscore_grade', 'ecoscore_score', 'carbon_footprint_100g', 'additives_count', 'sugar_ratio', 'energy_density', 'protein_ratio', 'macro_balance', 'healthy_score', 'log_energy_kcal_100g', 'log_fat_100g', 'log_sugars_100g', 'log_salt_100g']


In [2]:
target_columns = ['brand', 'allergens', 'ingredients_text', 'countries', 'additives']

for col in target_columns:
    print(f"\n{'='*60}")
    print(f"Column: {col}")
    print(f"{'='*60}")
    print(f"Data type: {df[col].dtype}")
    print(f"Total rows: {len(df)}")
    print(f"Non-null count: {df[col].notna().sum()}")
    print(f"Null count: {df[col].isna().sum()}")
    print(f"Unique values: {df[col].nunique()}")
    print(f"\nSample values (first 5 non-null):")
    for idx, val in enumerate(df[col].dropna().head(5).values):
        print(f"  {idx+1}. {repr(val)[:100]}")


Column: brand
Data type: object
Total rows: 5651
Non-null count: 5410
Null count: 241
Unique values: 2445

Sample values (first 5 non-null):
  1. 'La Casetta di Campagna'
  2. 'H-E-B Organics'
  3. 'DmBio'
  4. 'Diamond of california'
  5. 'Tree Of Life  Inc.'

Column: allergens
Data type: object
Total rows: 5651
Non-null count: 3672
Null count: 1979
Unique values: 364

Sample values (first 5 non-null):
  1. 'Nuts'
  2. 'Nuts'
  3. 'Nuts, Peanuts, Soybeans'
  4. 'Nuts'
  5. 'Nuts'

Column: ingredients_text
Data type: object
Total rows: 5651
Non-null count: 5267
Null count: 384
Unique values: 4911

Sample values (first 5 non-null):
  1. 'Italian: Mais'
  2. 'German: 99,5% Linsenmehl*, 0,5 % Meersalz. aus biologischer Landwirtschaft Kann Spuren von Soja und
  3. 'Almonds'
  4. 'Organic whole raw almonds'
  5. 'Almonds . soybean and/or peanut oil. sea salt.'

Column: countries
Data type: object
Total rows: 5651
Non-null count: 5639
Null count: 12
Unique values: 615

Sample values (first 

In [3]:
COUNTRY_NORMALIZATION = {
    "usa": "united states",
    "u.s.a.": "united states",
    "us": "united states",
    "united states of america": "united states",
    "uk": "united kingdom",
    "u.k.": "united kingdom",
    "england": "united kingdom",
    "scotland": "united kingdom",
    "wales": "united kingdom",
    "gb": "united kingdom",
    "germany": "germany",
    "deutschland": "germany",
    "austria": "austria",
    "österreich": "austria",
    "france": "france",
    "espagne": "spain",
    "spain": "spain",
    "italy": "italy",
}

COMPANY_SUFFIXES = {
    "inc", "inc.", "sa", "s.a.", "gmbh", "srl", "s.r.l.",
    "ltd", "ltd.", "co.", "company", "ag", "kg"
}


In [4]:
def _basic_clean(text: str) -> str:
    """Common text cleaning applied to all columns."""
    text = re.sub(r'http\S+|www\S+', '', text)

    text = re.sub(r'\S+@\S+', '', text)

    text = re.sub(r'&[a-z]+;', '', text)

    text = re.sub(
        r'^(german|french|italian|spanish|english|portuguese|dutch|swedish|danish|'
        r'norwegian|polish|czech|hungarian|romanian|bulgarian|greek|russian|ukrainian|'
        r'turkish|arabic|hebrew|japanese|chinese|korean):\s*',
        '',
        text,
        flags=re.IGNORECASE
    )

    text = re.sub(r'\[|\]', '', text)

    text = re.sub(r'\s*[,;]\s*', ', ', text)
    text = re.sub(r'\s*\.\s*', '. ', text)

    text = re.sub(r'\.{2,}', '.', text)
    text = re.sub(r'\*{2,}', '*', text)

    text = re.sub(r'\s+', ' ', text)

    text = text.lower().strip()

    return text



In [5]:
def _clean_ingredients(text: str) -> str:
    """
    Ingredients: keep main content, drop parentheses with 'may contain' / 'traces' / 'contains'
    because they are often cross-contamination metadata.
    """
    text = re.sub(r'\([^)]*(may contain|traces|contains)[^)]*\)', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [6]:
def _normalize_brand(text: str) -> str:
    """
    Brand: split on commas, remove duplicates and company suffixes.
    Return a pipe-separated list of normalized brand names.
    """
    if not text:
        return ''

    parts = [p.strip() for p in re.split(r'[,;/]+', text) if p.strip()]
    cleaned_brands = []

    for p in parts:
        tokens = [t for t in p.split() if t not in COMPANY_SUFFIXES]
        if not tokens:
            continue
        brand_name = ' '.join(tokens)
        cleaned_brands.append(brand_name)

    seen = set()
    unique_brands = []
    for b in cleaned_brands:
        if b not in seen:
            seen.add(b)
            unique_brands.append(b)

    return ' | '.join(unique_brands)

In [7]:
def _normalize_allergens(text: str) -> str:
    """
    Allergens: split on commas/semicolons, normalize phrases into canonical tags.
    Output is a pipe-separated list of allergen tags for easy ML use.
    """
    if not text:
        return ''

    raw_tokens = re.split(r'[;,/]+', text)
    normalized = []

    for tok in raw_tokens:
        tok = tok.strip().lower()
        if not tok:
            continue

        tok = re.sub(r'\bmay contain\b', '', tok)
        tok = re.sub(r'\btraces of\b', '', tok)
        tok = re.sub(r'\bcontains\b', '', tok)
        tok = tok.strip()

        if not tok:
            continue

        if 'sulphur dioxide' in tok or 'sulfites' in tok or 'sulphites' in tok:
            label = 'sulphur_dioxide_sulphites'
        elif 'peanut' in tok:
            label = 'peanuts'
        elif 'nut' in tok and 'peanut' not in tok:
            label = 'tree_nuts'
        elif 'milk' in tok or 'lactose' in tok or 'dairy' in tok:
            label = 'milk'
        elif 'egg' in tok:
            label = 'eggs'
        elif 'soy' in tok or 'soya' in tok:
            label = 'soybeans'
        elif 'gluten' in tok or 'wheat' in tok or 'barley' in tok or 'rye' in tok:
            label = 'gluten'
        elif 'sesame' in tok:
            label = 'sesame'
        elif 'fish' in tok:
            label = 'fish'
        elif 'crustacean' in tok or 'shrimp' in tok or 'prawn' in tok or 'crab' in tok:
            label = 'crustaceans'
        elif 'mustard' in tok:
            label = 'mustard'
        elif 'celery' in tok:
            label = 'celery'
        elif 'lupin' in tok:
            label = 'lupin'
        else:
            label = tok

        normalized.append(label)

    seen = set()
    unique_labels = []
    for a in normalized:
        if a not in seen:
            seen.add(a)
            unique_labels.append(a)

    return ' | '.join(unique_labels)




In [8]:
def _normalize_countries(text: str) -> str:
    """
    Countries: split, normalize variants (usa -> united states), drop 'world',
    deduplicate. Return pipe-separated list.
    """
    if not text:
        return ''

    parts = [p.strip().lower() for p in re.split(r'[;,/]+', text) if p.strip()]
    normalized = []

    for p in parts:
        p = re.sub(r'\s+', ' ', p)
        p = COUNTRY_NORMALIZATION.get(p, p)

        if p in {'world', 'en:world'}:
            continue

        normalized.append(p)

    seen = set()
    unique = []
    for c in normalized:
        if c not in seen:
            seen.add(c)
            unique.append(c)

    return ' | '.join(unique)


In [9]:
def _normalize_additives(text: str) -> str:
    """
    Additives: parse list-like strings and extract E-codes.
    Output: space-separated list of unique additive codes (e.g., 'e150d e621').
    """
    if not text:
        return ''

    s = text.strip()
    items = []

    if s.startswith('[') and s.endswith(']'):
        try:
            parsed = ast.literal_eval(s)
            if isinstance(parsed, str):
                items = [parsed]
            elif isinstance(parsed, (list, tuple)):
                items = [str(x) for x in parsed]
            else:
                items = [s]
        except Exception:
            items = re.split(r'[;,]+', s)
    else:
        items = re.split(r'[;,]+', s)

    codes = []

    for item in items:
        item = str(item).lower()
        item = item.strip()
        if not item:
            continue
        m = re.search(r'(e\s*\d+[a-z]?)', item, flags=re.IGNORECASE)
        if m:
            code = m.group(1).lower().replace(' ', '')
            codes.append(code)

    codes = sorted(set(codes))

    return ' '.join(codes)


In [10]:
def preprocess_text(text: Optional[str], column_type: str = 'general') -> str:
    """
    Column-aware text preprocessing tailored to this dataset.

    Args:
        text: raw cell value
        column_type: one of 'brand', 'allergens', 'ingredients_text',
                     'countries', 'additives', or 'general'.

    Returns:
        Cleaned / normalized text suitable for downstream ML.
    """
    if pd.isna(text) or text is None:
        return ''

    text = str(text).strip()
    if not text:
        return ''

    base = _basic_clean(text)

    if column_type == 'brand':
        return _normalize_brand(base)

    if column_type == 'allergens':
        return _normalize_allergens(base)

    if column_type == 'ingredients_text':
        return _clean_ingredients(base)

    if column_type == 'countries':
        return _normalize_countries(base)

    if column_type == 'additives':
        return _normalize_additives(base)

    return base

In [11]:
target_columns = ['brand', 'allergens', 'ingredients_text', 'countries', 'additives']

for col in target_columns:
    new_col_name = f'{col}_cleaned'
    print(f"Processing {col}...")
    df[new_col_name] = df[col].apply(lambda x: preprocess_text(x, column_type=col))
    print(f"  ✓ Created {new_col_name}")

Processing brand...
  ✓ Created brand_cleaned
Processing allergens...
  ✓ Created allergens_cleaned
Processing ingredients_text...
  ✓ Created ingredients_text_cleaned
Processing countries...
  ✓ Created countries_cleaned
Processing additives...
  ✓ Created additives_cleaned


In [12]:
comparison_cols = []
for col in target_columns:
    comparison_cols.extend([col, f'{col}_cleaned'])

print("Sample comparison (first 5 rows with non-empty values):")
print("\n" + "="*100)

sample_df = df[comparison_cols].head(10)
for idx, row in sample_df.iterrows():
    has_data = False
    for col in target_columns:
        if pd.notna(row[col]) and str(row[col]).strip():
            has_data = True
            break
    
    if has_data:
        print(f"\nRow {idx}:")
        for col in target_columns:
            orig = row[col]
            clean = row[f'{col}_cleaned']
            if pd.notna(orig) and str(orig).strip():
                print(f"  {col}:")
                print(f"    Original: {repr(str(orig)[:70])}")
                print(f"    Cleaned:  {repr(clean[:70])}")

Sample comparison (first 5 rows with non-empty values):


Row 0:
  brand:
    Original: 'La Casetta di Campagna'
    Cleaned:  'la casetta di campagna'
  ingredients_text:
    Original: 'Italian: Mais'
    Cleaned:  'mais'
  countries:
    Original: 'Italy'
    Cleaned:  'italy'
  additives:
    Original: '[]'
    Cleaned:  ''

Row 1:
  brand:
    Original: 'H-E-B Organics'
    Cleaned:  'h-e-b organics'
  countries:
    Original: 'United States'
    Cleaned:  'united states'
  additives:
    Original: '[]'
    Cleaned:  ''

Row 2:
  brand:
    Original: 'DmBio'
    Cleaned:  'dmbio'
  ingredients_text:
    Original: 'German: 99,5% Linsenmehl*, 0,5 % Meersalz. aus biologischer Landwirtsc'
    Cleaned:  '99, 5% linsenmehl*, 0, 5 % meersalz. aus biologischer landwirtschaft k'
  countries:
    Original: 'Germany'
    Cleaned:  'germany'
  additives:
    Original: '[]'
    Cleaned:  ''

Row 3:
  brand:
    Original: 'Diamond of california'
    Cleaned:  'diamond of california'
  allergens:

In [13]:
print("Preprocessing Summary Statistics")
print("="*70)

for col in target_columns:
    original = df[col]
    cleaned = df[f'{col}_cleaned']
    
    orig_non_null = original.notna().sum()
    orig_non_empty = (original.astype(str).str.strip() != '').sum()
    clean_non_empty = (cleaned.str.strip() != '').sum()
    
    print(f"\n{col}:")
    print(f"  Original non-null values: {orig_non_null}")
    print(f"  Original non-empty values: {orig_non_empty}")
    print(f"  Cleaned non-empty values: {clean_non_empty}")
    print(f"  Unique values (original): {original.nunique()}")
    print(f"  Unique values (cleaned): {cleaned.nunique()}")
    print(f"  Average length (original): {original.astype(str).str.len().mean():.1f}")
    print(f"  Average length (cleaned): {cleaned.str.len().mean():.1f}")

Preprocessing Summary Statistics

brand:
  Original non-null values: 5410
  Original non-empty values: 5651
  Cleaned non-empty values: 5410
  Unique values (original): 2445
  Unique values (cleaned): 2241
  Average length (original): 12.2
  Average length (cleaned): 11.7

allergens:
  Original non-null values: 3672
  Original non-empty values: 5651
  Cleaned non-empty values: 3672
  Unique values (original): 364
  Unique values (cleaned): 344
  Average length (original): 11.5
  Average length (cleaned): 11.6

ingredients_text:
  Original non-null values: 5267
  Original non-empty values: 5651
  Cleaned non-empty values: 5267
  Unique values (original): 4911
  Unique values (cleaned): 4876
  Average length (original): 245.6
  Average length (cleaned): 240.5

countries:
  Original non-null values: 5639
  Original non-empty values: 5651
  Cleaned non-empty values: 5636
  Unique values (original): 615
  Unique values (cleaned): 592
  Average length (original): 14.3
  Average length (clean

In [14]:
output_path = "preprocessed_foodfacts_phase2.csv"

df.to_csv(output_path, index=False, encoding="utf-8")